<a href="https://colab.research.google.com/github/granantuin/Model_vs_data_label/blob/master/Copia_de_Display_x_y_data_rawmodel_filter__wind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ml

    100% |████████████████████████████████| 102kB 4.7MB/s 


In [0]:
from sklearn.metrics import confusion_matrix ,accuracy_score,f1_score,recall_score
from sklearn.metrics import precision_recall_fscore_support
from pandas_ml import ConfusionMatrix
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
drive_4km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_4km_h24toh48_dir/"
drive_1km="/content/drive/My Drive/Colab Notebooks/model_vs_data/Coron_Mars_H24to48_dir/"
dr=[drive_4km,drive_1km]

In [0]:
y_data=pd.read_excel(dr[1]+"y_data.xlsx")
x_data=pd.read_csv(dr[1]+"x_data",usecols=range(1,10))
y_sp=pd.read_excel(dr[1]+"y_sp.xlsx") # units=Km/h to m/s
y_sp["spd"]=y_sp["spd"]/3.6

In [0]:
NE=y_data[(y_data["value"]>22.5) & (y_data["value"]<67.5)]
NE["label"]="NE"
E=y_data[(y_data["value"]>67.5) & (y_data["value"]<112.5)]
E["label"]="E"
SE=y_data[(y_data["value"]>112.5) & (y_data["value"]<157.5)]
SE["label"]="SE"
S=y_data[(y_data["value"]>157.5) & (y_data["value"]<202.5)]
S["label"]="S"
SW=y_data[(y_data["value"]>202.5) & (y_data["value"]<247.5)]
SW["label"]="SW"
W=y_data[(y_data["value"]>247.5) & (y_data["value"]<292.5)]
W["label"]="W"
NW=y_data[(y_data["value"]>292.5) & (y_data["value"]<337.5)]
NW["label"]="NW"
N=y_data[(y_data["value"]>337.5) | (y_data["value"]<22.5)]
N["label"]="N"
winds=[NE,N,E,SE,S,SW,W,NW]
y_data=pd.concat(winds)
y_data=y_data.sort_values(by="date")


In [0]:
x_data["date"]=y_data["date"]

In [0]:
for c in ['0', '1', '2', '3', '4', '5', '6', '7', '8']:
  NE=x_data[(x_data[c]>22.5) & (x_data[c]<67.5)]
  NE["label"+c]="NE"
  E=x_data[(x_data[c]>67.5) & (x_data[c]<112.5)]
  E["label"+c]="E"
  SE=x_data[(x_data[c]>112.5) & (x_data[c]<157.5)]
  SE["label"+c]="SE"
  S=x_data[(x_data[c]>157.5) & (x_data[c]<202.5)]
  S["label"+c]="S"
  SW=x_data[(x_data[c]>202.5) & (x_data[c]<247.5)]
  SW["label"+c]="SW"
  W=x_data[(x_data[c]>247.5) & (x_data[c]<292.5)]
  W["label"+c]="W"
  NW=x_data[(x_data[c]>292.5) & (x_data[c]<337.5)]
  NW["label"+c]="NW"
  N=x_data[(x_data[c]>337.5) | (x_data[c]<22.5)]
  N["label"+c]="N"
  winds=[NE,N,E,SE,S,SW,W,NW]
  x_data=pd.concat(winds)
  x_data=x_data.sort_values(by="date")
  
  

In [0]:
x_data=x_data.iloc[:, 10:]

In [0]:
x_data

In [0]:
for label in x_data.columns:
  print(label)
  index=["E","N","NE","NW","S","SE","SW","W"]
  #print(pd.DataFrame(confusion_matrix(y_data["label"],x_data[label] ), index=index, columns=index))
  print(ConfusionMatrix(y_data["label"],x_data[label]))
  print("****************")
  print("Accuracy=","{:.2%}".format(accuracy_score(y_data["label"], x_data[label])))
  print("****************")
  results= precision_recall_fscore_support(y_data["label"], x_data[label], average=None, )
  df=pd.DataFrame({"Precision":results[0],"Recall":results[1],"F1":results[2],"W_DIR":index})
  df=df.set_index("W_DIR")
  print(df)
  print("*******************************************************")
  print("Average precision =","{:.2%}".format(df["Precision"].mean()))
  print("Average recall =","{:.2%}".format(df["Recall"].mean()))
  results= precision_recall_fscore_support(y_data["label"], x_data[label], average="micro", )
  print("Precision micro=","{:.2%}".format(results[0]))
  print("Recall micro=","{:.2%}".format(results[1]))
  print("*******************************************************")
  

**Filteing data using spd(m*s) thershold**

In [0]:
y_sp["spd"].plot(kind="hist",bins=9)

In [0]:
x_data["date"]=y_data["date"]

In [0]:
x_data=x_data.set_index("date")
y_data=y_data.set_index("date")
y_sp=y_sp.set_index("date")
tot=pd.concat([x_data,y_data,y_sp],axis=1).dropna()

In [0]:
x_data=tot[tot["spd"]>2].iloc[:, 0:9]

In [0]:
y_data=tot[tot["spd"]>2].iloc[:, 10:11]

In [0]:
for label in x_data.columns:
  print(label)
  index=["E","N","NE","NW","S","SE","SW","W"]
  #print(pd.DataFrame(confusion_matrix(y_data["label"],x_data[label] ), index=index, columns=index))
  print(ConfusionMatrix(y_data["label"],x_data[label]))
  print("****************")
  print("Accuracy=","{:.2%}".format(accuracy_score(y_data["label"], x_data[label])))
  print("****************")
  results= precision_recall_fscore_support(y_data["label"], x_data[label], average=None, )
  df=pd.DataFrame({"Precision":results[0],"Recall":results[1],"F1":results[2],"W_DIR":index})
  df=df.set_index("W_DIR")
  print(df)
  print("*******************************************************")
  print("Average precision =","{:.2%}".format(df["Precision"].mean()))
  print("Average recall =","{:.2%}".format(df["Recall"].mean()))
  results= precision_recall_fscore_support(y_data["label"], x_data[label], average="micro", )
  print("Precision micro=","{:.2%}".format(results[0]))
  print("Recall micro=","{:.2%}".format(results[1]))
  print("*******************************************************")
  